## **Описание задачи и датасета**

Датасет Titanic описывает пассажиров парохода «Титаник», затонувшего в 1912 году. Необходимо решить задачу бинарной классификации: по данным о пассажире нужно предсказать, выжил он или нет.

Переменные датасета titanic:

Проект решает задачу бинарной классификации на датасете Titanic: по информации о пассажире нужно предсказать, выжил он в катастрофе или нет (`Survived`: 1 - выжил, 0 - погиб). Данные описывают 891 пассажира лайнера «Титаник» с признаками пола, возраста, класса билета, стоимости билета, каюты и порта посадки.  

Основные поля датасета:

- `PassengerId` - уникальный идентификатор пассажира (служебный);
- `Survived` - целевая переменная (1 - выжил, 0 - погиб);
- `Pclass` - класс билета (1 - высший, 2 - средний, 3 - низший);
- `Name` - полное имя пассажира;
- `Sex` - пол (`male` / `female`);
- `Age` - возраст (часть значений отсутствует);
- `SibSp` - число братьев/сестер и супругов на борту;
- `Parch` - число родителей и детей на борту;
- `Ticket` - номер билета;
- `Fare` - стоимость билета;
- `Cabin` - номер каюты;
- `Embarked` - порт посадки (`S`, `C`, `Q`).

In [824]:
# импорт необходимых библиотек
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from dotenv import load_dotenv
import os

In [825]:
# загрузка параметров из .env файла
load_dotenv("envs/.params.env", override=True)

TRAIN_DATA_URL = os.getenv("TRAIN_DATA_URL")
TEST_DATA_URL = os.getenv("TEST_DATA_URL")
VAL_SIZE = float(os.getenv("VAL_SIZE"))
EPOCHS = int(os.getenv("EPOCHS"))
HIDDEN_SIZE = int(os.getenv("MODEL_PARAMETER_1"))